In [42]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
# 加载.env文件中的环境变量
load_dotenv()

# 从环境变量获取OpenRouter API密钥
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

# 初始化ChatOpenAI模型，指定base_url为OpenRouter的API端点
llm = ChatOpenAI(
    model="deepseek/deepseek-chat-v3-0324:free",  # 例如 "openai/gpt-3.5-turbo"
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1"
)

# 使用模型进行聊天
# response = llm.invoke("你好，介绍一下自己。")
# print(response)

In [43]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings 

# 2. Prompts - 创建提示模板
prompt_template = PromptTemplate(
    input_variables=["product"],
    template="请为{product}写一个简短的广告语。"
)

# 3. Chains - 创建简单的链
simple_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

# 4. Memory - 创建记忆
memory = ConversationBufferMemory(return_messages=True)

# 5. Tools - 加载工具
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# 6. Agents - 初始化代理
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

# 7. Indexes - 创建索引(检索增强生成RAG的基础)
loader = TextLoader("example_data.txt")
embedding_api_key = os.getenv("TUZI_API_KEY")
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=embedding_api_key,
    openai_api_base="https://api.tu-zi.com/v1",
    tiktoken_enabled=True
)
index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])

# 使用示例

# 使用简单的链
ad_result = simple_chain.run(product="智能手表")
print(f"广告语生成结果: {ad_result}")

# 使用代理(可以使用工具和记忆)
agent_result = agent.run("谁是阿尔伯特·爱因斯坦？他的质能方程是什么？")
print(f"代理回答结果: {agent_result}")

# 使用索引进行检索问答
query_result = index.query("文档中最重要的主题是什么？",llm=llm)
print(f"检索问答结果: {query_result}")

/Users/childhoodandy/.pyenv/versions/3.11.3/lib/python3.11/site-packages/langchain/indexes/vectorstore.py:171: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


广告语生成结果: 1. **"时刻互联，健康随行——你的腕上智能管家！"**  
2. **"科技轻触手腕，未来触手可及。"**  
3. **"更懂你的心跳，更懂你的生活。"**  
4. **"一表在手，运动、通讯、生活全拥有！"**  
5. **"全天候续航，全方位守护——智慧生活，抬手即见。"**  

（根据产品特点如健康监测、长续航或潮流设计可灵活调整～）


> Entering new AgentExecutor chain...
Thought: 我需要查询阿尔伯特·爱因斯坦的基本信息和质能方程的相关内容。
Action: wikipedia  
Action Input: 阿尔伯特·爱因斯坦  

Observation
Observation: No good Wikipedia Search Result was found
Thought:由于“阿尔伯特·爱因斯坦”的中文搜索结果未找到，我将切换到英文搜索来获取相关信息。  
Action: wikipedia  
Action Input: Albert Einstein  


Observation: Page: Albert Einstein
Summary: Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is best known for developing the theory of relativity. Einstein also made important contributions to quantum mechanics. His mass–energy equivalence formula E = mc2, which arises from special relativity, has been called "the world's most famous equation". He received the 1921 Nobel Prize in Physics for his services to theoretical physics, and especially for his discovery of t

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `总结：阿尔伯特·爱因斯坦（Albert Einstein）是出生于德国的理论物理学家，他最著名的贡献包括发展狭义相对论和广义相对论，以及提出质能方程 \( E=mc^2 \)。这个方程表明质量和能量可以互相转化，并成为物理学中最著名的方程之一。爱因斯坦因其对理论物理学的贡献，特别是在光电效应方面的发现，而荣获1921年的诺贝尔物理学奖。  

此外，他在职业生涯中还对量子力学、统计力学等领域做出了重要贡献。他曾在美国度过晚年，并成为美国公民，以躲避纳粹对犹太人的迫害。  

最终答案：阿尔伯特·爱因斯坦（Albert Einstein）是一位著名的德国出生的理论物理学家，以创立相对论等理论闻名。他的质能方程是 \( E = mc^2 \)，这表明质量和能量之间的等价关系。爱因斯坦因光电效应理论获得1921年诺贝尔物理学奖。`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 